**This notebook is an edited approach to the df splitting and functions used in notebook 5_. It focuses on getting Spotify song data based on Kaggle dataset (same source as the larger one for artists used in notebook 4_) as well as using spotipy to pull data on any unmatched songs. This turned into scratch work. TLDR: Go to notebook 7_ for the actual final approach to getting and pickling song data.**

In [1]:
# Basics 
import pandas as pd
import numpy as np

# Text Cleaning
import re

#Spotify
# Source: https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time 

# Importing dfs with heavy processing
import pickle

In [2]:
# import peloton data
# Thank you to okaykristinakay on Reddit for the data!
# Source: https://www.reddit.com/r/pelotoncycle/comments/m18xnr/peloton_class_list_march_update/
df = pd.read_excel('../../../data/original_datasets/AGF_Peloton Classes March.xlsx')
df.head()

,classId,className,classDescription,classDifficulty,classDuration,classType,classLength,classLocation,classOriginalAirdate,classRating,classRatingCount,instructorName,instructorBio,classEquipment,classSongs,classArtists,classUrl
0,7f66378211c9476b9b5619bf989f91d0,20 min Peace Meditation,A guided meditation that focuses on cultivatin...,4.3333,20,Meditation,23,psny-studio-2,2021-09-03 13:25:00,0.9847,131,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...",Yoga Block,Meditation 22,RIOPY,https://members.onepeloton.com/classes/bootcam...
1,54ac61803b364b2fa8378acd9f593cdb,15 min Bodyweight Strength,"No equipment, no problem. Join us for a high-e...",5.7755,15,Strength,19,psny-studio-3,2021-09-03 13:19:00,0.9933,297,Olivia Amato,"Born and raised in New York, Olivia grew up pl...",Workout Mat,"California Gurls,Let's Get Loud,Let It Rock (f...","Katy Perry,Snoop Dogg,Jennifer Lopez,Kevin Rud...",https://members.onepeloton.com/classes/bootcam...
2,c75fd4831573483c9d45739aae11d083,20 min Focus Flow: Lower Body,This yoga flow class focuses on poses that eng...,4.3664,20,Yoga,23,psny-studio-2,2021-09-03 12:51:00,1.0000,159,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...","Yoga Blanket,Yoga Block,Yoga Mat","Interlude No 1,Oceansize,She Just Likes To Fig...","James Vincent McMorrow,Oh Wonder,Four Tet,Grim...",https://members.onepeloton.com/classes/bootcam...
3,470086936f7a4723ab5a53cb80b571ff,45 min Pop Bootcamp,Split your workout 50/50 between cardio on the...,7.8312,45,Tread Bootcamp,50,psny-studio-4,2021-09-03 11:56:00,0.9737,152,Olivia Amato,"Born and raised in New York, Olivia grew up pl...","Workout Mat,Medium Weights",34+35 (Remix) (feat. Doja Cat & Megan Thee Sta...,"Ariana Grande,Doja Cat,Megan Thee Stallion,Jus...",https://members.onepeloton.com/classes/bootcam...
4,9680a817bf2149d2b91990c87166a400,20 min Pop Ride,We dare you not to dance as you ride to all th...,7.4000,20,Cycling,24,uk,2021-09-03 07:52:00,1.0000,82,Sam Yo,Sam is a pro at many things but shines when it...,None,"Señorita,Marry You,Irreplaceable,What Do You M...","Justin Timberlake,Bruno Mars,Beyoncé,Justin Bi...",https://members.onepeloton.com/classes/bootcam...


In [3]:
# Opening pickle files
# Use classId where you can as key for merging
base_clean_df = pd.read_pickle("../../../data/pickled_dfs/base_clean_df.pkl")
artist_df = pd.read_pickle("../../../data/pickled_dfs/artist_df.pkl")
artist_id_class_count = pd.read_pickle("../../../data/pickled_dfs/larger_spot_artist_id_class_count.pkl")
songs_df = pd.read_pickle("../../../data/pickled_dfs/songs_df.pkl")
song_class_count_df = pd.read_pickle("../../../data/pickled_dfs/song_class_count_df.pkl")
song_class_count_df.rename(columns={'Artist':'Song'}, inplace=True) # Quick clean rename column
class_type_reorg_df = pd.read_pickle("../../../data/pickled_dfs/class_type_reorg_df.pkl")
new_class_types_with_artists = pd.read_pickle("../../../data/pickled_dfs/new_class_types_with_artists.pkl")
# artist_id_class_count has 186 rows with 0LyfQWJT6nXafLPZqxe9Of - it means Various Artists
# and 40 errors

In [4]:
base_clean_df
# Difference from df:
# Dropped null 'instructorBio' rows - only 9
# Dropped rows with null classSongs/classArtists
# Dropped row value with classSongs as float (1 row)
# Populate rows with classType Cycling and classEquipment None to classEquipment = Bike

,classId,className,classDescription,classDifficulty,classDuration,classType,classLength,classLocation,classOriginalAirdate,classRating,classRatingCount,instructorName,instructorBio,classEquipment,classSongs,classArtists,classUrl
0,7f66378211c9476b9b5619bf989f91d0,20 min Peace Meditation,A guided meditation that focuses on cultivatin...,4.3333,20,Meditation,23,psny-studio-2,2021-09-03 13:25:00,0.9847,131,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...",Yoga Block,Meditation 22,RIOPY,https://members.onepeloton.com/classes/bootcam...
1,54ac61803b364b2fa8378acd9f593cdb,15 min Bodyweight Strength,"No equipment, no problem. Join us for a high-e...",5.7755,15,Strength,19,psny-studio-3,2021-09-03 13:19:00,0.9933,297,Olivia Amato,"Born and raised in New York, Olivia grew up pl...",Workout Mat,"California Gurls,Let's Get Loud,Let It Rock (f...","Katy Perry,Snoop Dogg,Jennifer Lopez,Kevin Rud...",https://members.onepeloton.com/classes/bootcam...
2,c75fd4831573483c9d45739aae11d083,20 min Focus Flow: Lower Body,This yoga flow class focuses on poses that eng...,4.3664,20,Yoga,23,psny-studio-2,2021-09-03 12:51:00,1.0000,159,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...","Yoga Blanket,Yoga Block,Yoga Mat","Interlude No 1,Oceansize,She Just Likes To Fig...","James Vincent McMorrow,Oh Wonder,Four Tet,Grim...",https://members.onepeloton.com/classes/bootcam...
3,470086936f7a4723ab5a53cb80b571ff,45 min Pop Bootcamp,Split your workout 50/50 between cardio on the...,7.8312,45,Tread Bootcamp,50,psny-studio-4,2021-09-03 11:56:00,0.9737,152,Olivia Amato,"Born and raised in New York, Olivia grew up pl...","Workout Mat,Medium Weights",34+35 (Remix) (feat. Doja Cat & Megan Thee Sta...,"Ariana Grande,Doja Cat,Megan Thee Stallion,Jus...",https://members.onepeloton.com/classes/bootcam...
4,9680a817bf2149d2b91990c87166a400,20 min Pop Ride,We dare you not to dance as you ride to all th...,7.4000,20,Cycling,24,uk,2021-09-03 07:52:00,1.0000,82,Sam Yo,Sam is a pro at many things but shines when it...,Bike,"Señorita,Marry You,Irreplaceable,What Do You M...","Justin Timberlake,Bruno Mars,Beyoncé,Justin Bi...",https://members.onepeloton.com/classes/bootcam...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16743,a144fcf1595740caa610f6484deb3cdb,20 min Funk Fun Run,This 20 minute Fun Run is a musically driven c...,6.4567,20,Running,23,nyc-gw-tread,14/09/2018 10:22,0.9933,150,Becs Gentry,Becs joins Peloton from London as an accomplis...,None,"Man Funk (feat. Leron Thomas),If It Ain't Funk...","Guts,Leron Thomas,Soopasoul,FDEL,Basement Frea...",https://members.onepeloton.com/classes/bootcam...
16744,bd89a56c884b48e59ce9c43ab4f4d86f,45 min 90s Pop Ride,The 90s were a booming and inventive time in m...,8.1481,45,Cycling,49,nyc,13/09/2018 23:19,0.9925,1877,Denis Morton,"Raised in Florida, schooled in Tennessee, stee...",Light Weights,"Too Close - Radio Edit,Motownphilly - Original...","Next,Boyz II Men,Blues Traveler,Eagle-Eye Cher...",https://members.onepeloton.com/classes/bootcam...
16745,e5a99c83d7fc4f0686c68b02e5932204,10 min Arms Toning,Join Jess as she takes you through this 10-min...,7.4028,10,Strength,11,nyc,13/09/2018 21:02,0.9978,4481,Jess King,Jess is a charismatic instructor with a boundl...,Light Weights,"Hustle,Day and Night,Just Got Paid,Waiting for...","Daphne Willis,Gizzle,Lo Air,Sigala,Ella Eyre,M...",https://members.onepeloton.com/classes/bootcam...
16746,c57e123004d04df1bfae94c5e4399bb3,45 min 90s Ride,The 90s were a booming and inventive time in m...,8.3982,45,Cycling,49,nyc,13/09/2018 11:53,0.9922,2307,Emma Lovewell,Emma Lovewell is a Martha’s Vineyard native an...,Light Weights,"Show Me Love - Radio Version,Don't Speak,Gonna...","Robyn,No Doubt,C & C Music Factory,Technotroni...",https://members.onepeloton.com/classes/bootcam...


Note:
classId = 8840f82cdc624aa28a5d1babc51a3916 is the one you need to watch out for (has float classSong)

In [5]:
# Check lengths of dfs
print(f'artist_df: {len(artist_df)}')
print(f'artist_id_class_count: {len(artist_id_class_count)}')
print(f'songs_df: {len(songs_df)}')
print(f'song_class_count_df: {len(song_class_count_df)}')
print(f'class_type_reorg_df: {len(class_type_reorg_df)}')
print(f'new_class_types_with_artists: {len(new_class_types_with_artists)}')
# No inconsistencies where there shouldn't be

artist_df: 16748
artist_id_class_count: 10941
songs_df: 16748
song_class_count_df: 29904
class_type_reorg_df: 16748
new_class_types_with_artists: 16748


#### Notes:
- artist_df has a row for each of the 16748 classes with the lists of artists in that class and 10943 columns
    - column for classId, classArtists (from base_clean_df)
    - then each of the 10941 artists

- artist_id_class_count has 10941 rows for each of the artists with their class count, followers, genres, popularity
    - keep in mind nulls not cleaned yet...
    - keep them in there until you decide what you're doing with the data
    - can clean after merging with final data set


### Functions

In [6]:
# getSongId2 returns the FULL songid
# getSongId (the original) only returned the id after the first 15 characters.
# Something went wrong here and getting track features from these Ids were erroring out
def getSongId2(song):
    searchterm = song
    try:
        result = sp.search(searchterm)
        songid = result['tracks']['items'][0]['uri']
    except: # if do not find match, error
        return "error"
        pass
    else: # if do find match, return id
        return songid

In [7]:
# # Accounts for errors/nans -- try/except sections
# # Creaspotify:track:5yAcJtUiOFTdvePkHb4hs8te function to pull out track features
# # Source: https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37
# # Edited to put track information as additional row in track_df
# def getTrackFeatures(id):
# #     id = 'spotify:track:' + id
#     try:
#         meta = sp.track(id)
#         features = sp.audio_features(id)

#         # meta
#         id_searched = id
#         name = meta['name']
#         album = meta['album']['name']
#         artist = meta['album']['artists'][0]['name']
#         release_date = meta['album']['release_date']
#         length = meta['duration_ms']
#         popularity = meta['popularity']

#         # features
#         acousticness = features[0]['acousticness']
#         danceability = features[0]['danceability']
#         energy = features[0]['energy']
#         instrumentalness = features[0]['instrumentalness']
#         liveness = features[0]['liveness']
#         loudness = features[0]['loudness']
#         speechiness = features[0]['speechiness']
#         tempo = features[0]['tempo']
#         time_signature = features[0]['time_signature']

#         track = [id_searched, name, album, artist, release_date, length, popularity, danceability, acousticness, 
#                  danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
#     except: # if do not find match, pass
#         pass
#     else: # if do find match, return id
#         track_df.loc[(len(track_df) +1)] = track
#         return track_df

In [8]:
# getTrackFeatures2 pulls out more info to match kaggle dataset
# difference is there is not logic to pull multiple artists and their ids; this will just have artist name
# Create function to pull out track features
# Source: https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37
# Edited to put track information as additional row in track_df_2
def getTrackFeatures2(id):
    try:
        meta = sp.track(id)
        features = sp.audio_features(id)

        # meta
        id_searched = id
        name = meta['name']
        album = meta['album']['name']
        artist = meta['album']['artists'][0]['name'] # this will be unique feature to set apart for kaggle dataset
        release_date = meta['album']['release_date']
        duration_ms = meta['duration_ms'] # renamed to match kaggle
        explicit = meta['explicit'] # added to match kaggle dataset
        popularity = meta['popularity']

        # features
        acousticness = features[0]['acousticness']
        danceability = features[0]['danceability']
        energy = features[0]['energy']
        instrumentalness = features[0]['instrumentalness']
        liveness = features[0]['liveness']
        loudness = features[0]['loudness']
        speechiness = features[0]['speechiness']
        tempo = features[0]['tempo']
        time_signature = features[0]['time_signature']
        key = features[0]['key']
        mode = features[0]['mode']
        valence = features[0]['valence']


        track = [id_searched, name, album, artist, release_date, duration_ms, explicit, popularity,
                 acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, 
                 tempo, time_signature, key, mode, valence]
    except: # if do not find match, pass
        pass
    else: # if do find match, return id
        track_df_2.loc[(len(track_df_2) +1)] = track
        return track_df_2

In [80]:
# Create df to hold Spotify track features returned from function below
track_df_2 = pd.DataFrame(columns = ['id_searched', 'name', 'album', 'artist', 'release_date', 
                                   'duration_ms', 'explicit', 'popularity', 'acousticness', 
                                   'danceability', 'energy', 'instrumentalness', 'liveness', 
                                   'loudness', 'speechiness', 'tempo', 'time_signature', 'key',
                                   'mode', 'valence'])

### Spotify API Work

In [10]:
# Spotify API set up
# Source: https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37
client_id = '4aac5b215c8a4fc591b506b71af7ebf5'
client_secret = '31d999b609424f569382b047ba828b00'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#### Before cleaning Song List try to match to Kaggle Spotify df for track IDs

In [11]:
# Import Kaggle Spotify track IDs for matching
# Source: https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks?select=data_by_artist_o.csv
kag2_spot_track_df = pd.read_csv('tracks.zip', compression='zip')
kag2_spot_track_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [12]:
# Add spotify:track: to ids in id column for searching/get track features
kag2_spot_track_df['id'] = [('spotify:track:'+ x) for x in kag2_spot_track_df['id']]

In [13]:
# Check work
kag2_spot_track_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,spotify:track:35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,spotify:track:021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,spotify:track:07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,spotify:track:08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,spotify:track:08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [14]:
# Rename Song column 'name' to prep for merge
song_class_count_df.rename(columns={'Song':'name'}, inplace=True)

In [15]:
song_class_count_spotid = song_class_count_df.merge(kag2_spot_track_df, on='name', how='left', indicator=True)

In [16]:
song_class_count_spotid['_merge'].value_counts()
# 60157 total rows now; duplicate rows added

both          41048
left_only     19109
right_only        0
Name: _merge, dtype: int64

In [17]:
# View duplicates
song_class_count_spotid[song_class_count_spotid['name'].duplicated(keep='first')].sort_values(by='name')['name']

26802                        #1
26803                        #1
25491                        #3
39944                       #41
30817           'Round Midnight
                  ...          
36414                       pov
8114     through the late night
10984           Échame La Culpa
38900           Über den Wolken
38901           Über den Wolken
Name: name, Length: 30253, dtype: object

In [18]:
# Drop duplicate rows
# Peloton classes use songs viewers know
# Take first spotify id matched with for 30253 rows
# Can also test this data set completey reliant on Spotify API searches...
# Would take forever...
song_class_count_spotid.drop_duplicates(subset='name', keep='first', inplace=True)

In [19]:
# Reset index post drops
song_class_count_spotid = song_class_count_spotid.reset_index(drop=True)

In [20]:
# Check length - back to original count, duplicates removed
song_class_count_spotid

,name,Class Count,id,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,_merge
0,,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,Weekend,2,spotify:track:67UJGroXxC22oxzqO8bozE,25.0,113107.0,0.0,['Eddie Cochran'],['1p0t3JtUTayV2wb1RGN9mO'],1962-01-01,0.738,...,-8.349,1.0,0.0312,0.4760,0.002770,0.294,0.959,98.779,4.0,both
2,Supalonely (feat. Gus Dapperton),1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,Play (Artful Dodger Mix - Main Mix Radio),1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,Light Your Ass On Fire (Club Mix) (feat. Pharr...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29899,Wish You Were Here (feat. Brandyn Burnette) [E...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
29900,Modern Jesus (Charli XCX Remix),2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
29901,Sandstorm - Komytea Radio Remix,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
29902,Just To Keep You Satisfied (Album Version),1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [21]:
leftover_songs = list(song_class_count_spotid[song_class_count_spotid['id'].isnull()]['name'])

In [22]:
leftover_songs

['',
 'Supalonely (feat. Gus Dapperton)',
 'Play (Artful Dodger Mix - Main Mix Radio)',
 'Light Your Ass On Fire (Club Mix) (feat. Pharrell)',
 'Moon and Star',
 "Let's Stay Together - 2015 Remastered Version",
 'There for You feat. Effie (Crush Club Remix)',
 'I Feel for You (Axwell Remix)',
 'I Like It - Dillon Francis Remix',
 'Din Daa Daa',
 'Beardo (Radio Edit)',
 'Ready For Your Love (CLOSE Ready For Your Dub) (feat. MNEK)',
 "When It's Over (Remastered)",
 'Mullholland',
 'Lose Sight',
 'O-H-I-O',
 'Piano Sonata No. 14 in C sharp minor',
 'Get Off Of My Cloud ((Original Single Mono Version))',
 'Turn The Light',
 'Another Brick in the Wall',
 ' Baby (Single Version)',
 'CoCo (MAKJ Remix)',
 'Get It Off (That Kid Chris Edit)',
 'Good Together - Alex Metric Remix',
 'Big Gucci',
 'Back From Hell (Remix)',
 'Summertime Sadness [Lana Del Rey vs. Cedric Gervais] - Cedric Gervais Remix',
 'I Will Survive (Album Version)',
 "Don't Let Go (Love) [Radio Edit]",
 'I Will Always Love You (

Clean leftover songs for Spotify API work
- DO NOT REMOVE PUNCTUATION

In [23]:
# Remove leading and trailing spaces from values
leftover_songs = [x.strip() for x in leftover_songs]

In [24]:
leftover_songs = sorted(leftover_songs)

In [25]:
# First value is a blank space
leftover_songs = leftover_songs[1:]

In [26]:
len(leftover_songs)

19108

#### Song Searches and Testing

In [27]:
# Create new df for songs and id capture
leftover_songs_df = pd.DataFrame(leftover_songs, columns=['song'])

# Create new column for Spotify Ids pulled through function
leftover_songs_df['spotifyId_man'] = ""

#### Testing First 1K

In [105]:
leftover_songs_df[:1001]

,song,spotifyId_man
0,"""(Mucho Mambo) Sway""",spotify:track:499en6xJ7unV8IluPBAYch
1,"""40 Day Dream""",spotify:track:4ek3pPdfvIbPxynmlisI0k
2,"""40""",spotify:track:0vtJ9Dq53hfXE3KyP1yPni
3,"""99""",spotify:track:7sLpSWxQazJzDVG6YGzlVs
4,"""Che gelida manina"" from La Bohème",spotify:track:6Px8dDwYhF0yogstmbr3sU
...,...,...
996,Applause (Bent Collective Club Mix),spotify:track:5hy7hLN6OgueCjyxTub7aF
997,Applause - DJ White Shadow Trap Remix,spotify:track:2OSDDyJMUiSvOHtDKBWag5
998,Applause - Empire Of The Sun Remix,spotify:track:4RLcgyT99Rctgc05rd4DkD
999,Applause - Viceroy Remix,spotify:track:1lLx3EEMZ4MFk0hbiEhSCu


In [81]:
leftover_songs_df['spotifyId_man'][:1001] = leftover_songs_df['song'][:1001].map(lambda a: getSongId2(a))
first_1k = leftover_songs_df[:1001]

In [82]:
first_1k_ids = first_1k['spotifyId_man']

In [83]:
first_1k_ids

0       spotify:track:499en6xJ7unV8IluPBAYch
1       spotify:track:4ek3pPdfvIbPxynmlisI0k
2       spotify:track:0vtJ9Dq53hfXE3KyP1yPni
3       spotify:track:7sLpSWxQazJzDVG6YGzlVs
4       spotify:track:6Px8dDwYhF0yogstmbr3sU
                        ...                 
996     spotify:track:5hy7hLN6OgueCjyxTub7aF
997     spotify:track:2OSDDyJMUiSvOHtDKBWag5
998     spotify:track:4RLcgyT99Rctgc05rd4DkD
999     spotify:track:1lLx3EEMZ4MFk0hbiEhSCu
1000    spotify:track:1ktEdYLczUPSNjHNGwBLIy
Name: spotifyId_man, Length: 1001, dtype: object

In [84]:
first_1k_ids.value_counts()

error                                   124
spotify:track:4srrfFBQgCoGTgV6ZzDe3c      2
spotify:track:5LPGZnsy07YNxeuTI1X3RT      2
spotify:track:1G54pa1a2jZQmOpTHh4nht      2
spotify:track:3JKgcAa7r07ocVWcV8bS0H      2
                                       ... 
spotify:track:1YMfkwjHHodR1rlTs2p6I6      1
spotify:track:6dDYckwvmMGZO1qB1tiuWv      1
spotify:track:0FKb4nOc59CoIaUDHUng4g      1
spotify:track:5RacpqELGv0C13HlrLGebK      1
spotify:track:4Dl4x1VMWScmqi7vceVnmj      1
Name: spotifyId_man, Length: 847, dtype: int64

In [85]:
# test success for first 1k before go through 19k again...
for song_id in first_1k_ids:
    if (song_id == 'error'):
        continue
    else:
        getTrackFeatures2(song_id)

In [90]:
track_df_first_1k = track_df_2.copy()

In [91]:
len(track_df_first_1k)

876

In [92]:
track_df_first_1k[track_df_first_1k['id_searched'].duplicated(keep=False)].sort_values(by='id_searched')
# Duplicated Ids...
# Don't Dedupe until all are done

,id_searched,name,album,artist,release_date,duration_ms,explicit,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,key,mode,valence
220,spotify:track:03enfjB0Jop0kRJx8AODM5,8 Letters - R3HAB Remix,8 Letters (Remixes),Why Don't We,2019-01-14,132232,False,50,0.2400,0.638,0.708,0.0000,0.3110,-5.001,0.0421,103.160,4,1,0,0.582
219,spotify:track:03enfjB0Jop0kRJx8AODM5,8 Letters - R3HAB Remix,8 Letters (Remixes),Why Don't We,2019-01-14,132232,False,50,0.2400,0.638,0.708,0.0000,0.3110,-5.001,0.0421,103.160,4,1,0,0.582
176,spotify:track:0ldLTYuqnHU92weLtw6W9m,3AM Eternal - Live at the S.S.L.,Solid State Logik 1,The KLF,2021-01-01,232411,False,49,0.0025,0.616,0.977,0.0124,0.9420,-6.357,0.0827,119.303,4,1,1,0.744
175,spotify:track:0ldLTYuqnHU92weLtw6W9m,3AM Eternal - Live at the S.S.L.,Solid State Logik 1,The KLF,2021-01-01,232411,False,49,0.0025,0.616,0.977,0.0124,0.9420,-6.357,0.0827,119.303,4,1,1,0.744
858,spotify:track:1G54pa1a2jZQmOpTHh4nht,Anything Can Happen (feat. Meek Mill),Ghetto Lenny's Love Songs,SAINt JHN,2019-08-23,182995,True,58,0.0236,0.674,0.814,0.0000,0.4010,-5.861,0.0692,138.017,4,2,1,0.182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,spotify:track:6n7rJUcwmQDestMYXoSoZF,All Cried Out (feat. Alex Newell) - Radio Edit,All Cried Out (feat. Alex Newell),Blonde,2015-04-10,170716,False,64,0.4280,0.657,0.757,0.0000,0.0728,-5.257,0.0584,122.056,4,0,0,0.672
164,spotify:track:7GqIDx2QVGOpd4r1fZaUUW,25 or 6 to 4 - 2002 Remaster,Chicago II,Chicago,1970-01-26,289800,False,65,0.3090,0.604,0.905,0.0632,0.0837,-7.059,0.0578,147.448,4,4,0,0.723
163,spotify:track:7GqIDx2QVGOpd4r1fZaUUW,25 or 6 to 4 - 2002 Remaster,Chicago II,Chicago,1970-01-26,289800,False,65,0.3090,0.604,0.905,0.0632,0.0837,-7.059,0.0578,147.448,4,4,0,0.723
710,spotify:track:7dgFG4y5nz5tV8lLceQAKo,Always Gonna Be (feat. Alex Mills) [Low Steppa...,Defected In The House Radio Show Episode 049 (...,Defected Radio,2017-05-05,267579,False,12,0.0288,0.748,0.926,0.0952,0.3540,-5.611,0.0365,123.007,4,9,0,0.344


#### 1K-2K

In [103]:
leftover_songs_df['spotifyId_man'][1001:2001]

1001    spotify:track:5YjujDEmrypAj85VhKW8E9
1002    spotify:track:1HFguX7RTd1GYkZzmJMX39
1003    spotify:track:0BHSEhacCgjzFmSJSCewND
1004    spotify:track:0djZ2ndRfAL69WYNra5jRC
1005                                   error
                        ...                 
1996    spotify:track:2LjkT4gu5wO4JdeEYl0fMY
1997    spotify:track:32GWSQbpg1Ea1MMvl8HN23
1998    spotify:track:2jgPmBI3gIZ5C8DBlmFFGT
1999    spotify:track:4PFhHd7DsNg0ke9Nmlxms5
2000    spotify:track:2bAD41U8RQh4Z7rJinqzNy
Name: spotifyId_man, Length: 1000, dtype: object

In [95]:
leftover_songs_df['spotifyId_man'][1001:2001] = leftover_songs_df['song'][1001:2001].map(lambda a: getSongId2(a))
songs_1k_to_2k = leftover_songs_df[1001:2001]

In [99]:
songs_1k_to_2k

,song,spotifyId_man
1001,Appreciate Me,spotify:track:5YjujDEmrypAj85VhKW8E9
1002,Appreciating Nature,spotify:track:1HFguX7RTd1GYkZzmJMX39
1003,Approaching Ends,spotify:track:0BHSEhacCgjzFmSJSCewND
1004,April (Come She Will),spotify:track:0djZ2ndRfAL69WYNra5jRC
1005,Aqualight with Sebastien Leger,error
...,...,...
1996,Blue (feat. Blue Ivy),spotify:track:2LjkT4gu5wO4JdeEYl0fMY
1997,Blue And Yellow,spotify:track:32GWSQbpg1Ea1MMvl8HN23
1998,Blue Eyed Soul (2017 Remaster),spotify:track:2jgPmBI3gIZ5C8DBlmFFGT
1999,Blue Eyes (Remastered 2003),spotify:track:4PFhHd7DsNg0ke9Nmlxms5


In [96]:
songs_1k_to_2k_ids = songs_1k_to_2k['spotifyId_man']

In [97]:
songs_1k_to_2k_ids

1001    spotify:track:5YjujDEmrypAj85VhKW8E9
1002    spotify:track:1HFguX7RTd1GYkZzmJMX39
1003    spotify:track:0BHSEhacCgjzFmSJSCewND
1004    spotify:track:0djZ2ndRfAL69WYNra5jRC
1005                                   error
                        ...                 
1996    spotify:track:2LjkT4gu5wO4JdeEYl0fMY
1997    spotify:track:32GWSQbpg1Ea1MMvl8HN23
1998    spotify:track:2jgPmBI3gIZ5C8DBlmFFGT
1999    spotify:track:4PFhHd7DsNg0ke9Nmlxms5
2000    spotify:track:2bAD41U8RQh4Z7rJinqzNy
Name: spotifyId_man, Length: 1000, dtype: object

In [100]:
songs_1k_to_2k_ids.value_counts()

error                                   124
spotify:track:4XVfNwP88Hn9kSkXVnxj2S      3
spotify:track:2g6V8u3EoMA1TAPHtwrRHc      2
spotify:track:1MfuskV4ODUp14C2dnyPgW      2
spotify:track:7hf89cT5FEmLV5E9fjrjG7      2
                                       ... 
spotify:track:72HKgOTKNpp3glpdLZc1yo      1
spotify:track:5OGfBbWmRRkDiZiJbu5WIr      1
spotify:track:5Jz5POp67L1TSM9FmdZSdR      1
spotify:track:2lmMpvoIqZWWFBcKoQ8KSC      1
spotify:track:43vI4vyL0hRk4rAIeQlbax      1
Name: spotifyId_man, Length: 841, dtype: int64

In [106]:
# Reset track_df_2
# Create df to hold Spotify track features returned from function below
track_df_2 = pd.DataFrame(columns = ['id_searched', 'name', 'album', 'artist', 'release_date', 
                                   'duration_ms', 'explicit', 'popularity', 'acousticness', 
                                   'danceability', 'energy', 'instrumentalness', 'liveness', 
                                   'loudness', 'speechiness', 'tempo', 'time_signature', 'key',
                                   'mode', 'valence'])

In [109]:
# test success for first 1k before go through 19k again...
for song_id in songs_1k_to_2k_ids:
    if (song_id == 'error'):
        continue
    else:
        getTrackFeatures2(song_id)

In [110]:
track_df_2

,id_searched,name,album,artist,release_date,duration_ms,explicit,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,key,mode,valence
1,spotify:track:5YjujDEmrypAj85VhKW8E9,Appreciate Me,Appreciate Me,Rack$tar Drvco,2020-12-28,180346,True,28,0.023700,0.694,0.421,0,0.1430,-12.885,0.3420,128.944,5,9,1,0.0905
2,spotify:track:1HFguX7RTd1GYkZzmJMX39,Appreciating Nature,Spiritual Nature Rain,#Sleepy Rain,2021-05-21,150000,False,0,0.000802,0.259,0.274,0.967,0.4070,-28.765,0.0867,110.964,4,8,1,0.0387
3,spotify:track:0BHSEhacCgjzFmSJSCewND,Approaching Ends,Slow Waves,Submerse,2014-07-25,205736,False,24,0.219000,0.668,0.487,0.0458,0.1590,-8.409,0.0824,173.963,3,5,0,0.1380
4,spotify:track:0djZ2ndRfAL69WYNra5jRC,April Come She Will,Sounds Of Silence,Simon & Garfunkel,1966-01-17,109386,False,61,0.863000,0.489,0.186,0.000152,0.2000,-16.237,0.0290,102.245,4,9,1,0.3490
5,spotify:track:71wT7aMCFPYfzutF66OLac,Aquaman,TALKING IS HARD,WALK THE MOON,2014-12-02,240000,False,44,0.510000,0.630,0.772,2.26e-05,0.0881,-6.986,0.0297,99.964,4,1,1,0.7210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,spotify:track:2LjkT4gu5wO4JdeEYl0fMY,Blue (feat. Blue Ivy),BEYONCÉ [Platinum Edition],Beyoncé,2014-11-24,266920,False,52,0.541000,0.459,0.435,6.51e-05,0.1190,-10.139,0.0640,90.567,4,1,1,0.0631
873,spotify:track:32GWSQbpg1Ea1MMvl8HN23,Blue and Yellow,The Used,The Used,2002-06-25,201426,False,53,0.029700,0.565,0.867,0,0.1470,-5.497,0.0333,115.006,4,2,0,0.2590
874,spotify:track:2jgPmBI3gIZ5C8DBlmFFGT,Blue Eyed Soul - 2017 Remaster,A.M. (Deluxe Edition),Wilco,1995,244853,False,16,0.267000,0.594,0.582,0.00744,0.1760,-7.301,0.0309,135.726,4,2,1,0.3440
875,spotify:track:4PFhHd7DsNg0ke9Nmlxms5,Blue Eyes - Remastered 2003,Jump Up!,Elton John,1982-01-01,206266,False,55,0.575000,0.518,0.229,0.00363,0.1230,-17.180,0.0270,106.665,3,7,0,0.1590


In [111]:
track_df_1k_to_2k = track_df_2.copy()

In [112]:
len(track_df_1k_to_2k)

876

In [113]:
track_df_1k_to_2k[track_df_1k_to_2k['id_searched'].duplicated(keep=False)].sort_values(by='id_searched')
# Duplicated Ids...
# Don't Dedupe until all are done

,id_searched,name,album,artist,release_date,duration_ms,explicit,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,key,mode,valence
6,spotify:track:0HZlND4giwzgolBpaNIRGV,Aquarius/Let The Sunshine In (The Flesh Failur...,The Age Of Aquarius,The 5th Dimension,1969,289293,False,59,0.44600,0.293,0.740,0.00151,0.0998,-6.098,0.0399,118.230,4,7,1,0.516
7,spotify:track:0HZlND4giwzgolBpaNIRGV,Aquarius/Let The Sunshine In (The Flesh Failur...,The Age Of Aquarius,The 5th Dimension,1969,289293,False,59,0.44600,0.293,0.740,0.00151,0.0998,-6.098,0.0399,118.230,4,7,1,0.516
231,spotify:track:0SIhsJvOPXxEm6fr2wNzWN,Baby,Comenzando la semana,Various Artists,2021-05-28,240013,False,0,0.16300,0.712,0.682,0,0.1130,-5.852,0.0556,160.494,4,7,1,0.484
204,spotify:track:0SIhsJvOPXxEm6fr2wNzWN,Baby,Comenzando la semana,Various Artists,2021-05-28,240013,False,0,0.16300,0.712,0.682,0,0.1130,-5.852,0.0556,160.494,4,7,1,0.484
820,spotify:track:0UM8xa2o8jHCpvqQEq6Rfw,Blame (feat. John Newman) - Jacob Plant Remix,Blame (Remixes) (feat. John Newman),Calvin Harris,2014-11-10,296106,False,39,0.00269,0.653,0.893,0.000622,0.6190,-3.869,0.0347,127.974,4,0,0,0.566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,spotify:track:7aLemVaGw5bt4u7wPXm7Lm,As Your Friend,As Your Friend,Afrojack,2013-01-01,240004,True,46,0.04670,0.654,0.761,0,0.1040,-4.404,0.0678,128.033,4,6,1,0.805
572,spotify:track:7g4I83EkCeMo779gSo4zKW,"Beethoven: Symphony No. 5 in C Minor, Op. 67: ...",50 Best Classics,Various Artists,2010-02-26,395680,False,41,0.98300,0.328,0.232,0.843,0.2180,-15.058,0.0444,85.821,4,0,0,0.201
570,spotify:track:7g4I83EkCeMo779gSo4zKW,"Beethoven: Symphony No. 5 in C Minor, Op. 67: ...",50 Best Classics,Various Artists,2010-02-26,395680,False,41,0.98300,0.328,0.232,0.843,0.2180,-15.058,0.0444,85.821,4,0,0,0.201
467,spotify:track:7hf89cT5FEmLV5E9fjrjG7,beamer boy,beamer boy,Lil Peep,2017-06-08,203373,True,77,0.05090,0.743,0.732,1.77e-05,0.2270,-4.511,0.1040,123.994,4,8,1,0.339


#### 2K-3K

In [115]:
for num in range(3,4):
    search_start = ((num-1)*1000)+1
    search_end = (num*1000)+1
    print(f'{search_start}:{search_end}')

2001:3001


In [117]:
for num in range(3,4):
    search_start = ((num-1)*1000)+1
    search_end = (num*1000)+1
#     leftover_songs_df['spotifyId_man'][search_start:search_end] = leftover_songs_df['song'][search_start:search_end].map(lambda a: getSongId2(a))
    songs_{search_start}_{search_end} = leftover_songs_df[search_start:search_end]
    return songs_{search_start}_{search_end}

SyntaxError: invalid syntax (<ipython-input-117-3970739a3976>, line 5)

In [103]:
leftover_songs_df['spotifyId_man'][1001:2001]

1001    spotify:track:5YjujDEmrypAj85VhKW8E9
1002    spotify:track:1HFguX7RTd1GYkZzmJMX39
1003    spotify:track:0BHSEhacCgjzFmSJSCewND
1004    spotify:track:0djZ2ndRfAL69WYNra5jRC
1005                                   error
                        ...                 
1996    spotify:track:2LjkT4gu5wO4JdeEYl0fMY
1997    spotify:track:32GWSQbpg1Ea1MMvl8HN23
1998    spotify:track:2jgPmBI3gIZ5C8DBlmFFGT
1999    spotify:track:4PFhHd7DsNg0ke9Nmlxms5
2000    spotify:track:2bAD41U8RQh4Z7rJinqzNy
Name: spotifyId_man, Length: 1000, dtype: object

In [95]:
leftover_songs_df['spotifyId_man'][1001:2001] = leftover_songs_df['song'][1001:2001].map(lambda a: getSongId2(a))
songs_1k_to_2k = leftover_songs_df[1001:2001]

In [99]:
songs_1k_to_2k

,song,spotifyId_man
1001,Appreciate Me,spotify:track:5YjujDEmrypAj85VhKW8E9
1002,Appreciating Nature,spotify:track:1HFguX7RTd1GYkZzmJMX39
1003,Approaching Ends,spotify:track:0BHSEhacCgjzFmSJSCewND
1004,April (Come She Will),spotify:track:0djZ2ndRfAL69WYNra5jRC
1005,Aqualight with Sebastien Leger,error
...,...,...
1996,Blue (feat. Blue Ivy),spotify:track:2LjkT4gu5wO4JdeEYl0fMY
1997,Blue And Yellow,spotify:track:32GWSQbpg1Ea1MMvl8HN23
1998,Blue Eyed Soul (2017 Remaster),spotify:track:2jgPmBI3gIZ5C8DBlmFFGT
1999,Blue Eyes (Remastered 2003),spotify:track:4PFhHd7DsNg0ke9Nmlxms5


In [96]:
songs_1k_to_2k_ids = songs_1k_to_2k['spotifyId_man']

In [97]:
songs_1k_to_2k_ids

1001    spotify:track:5YjujDEmrypAj85VhKW8E9
1002    spotify:track:1HFguX7RTd1GYkZzmJMX39
1003    spotify:track:0BHSEhacCgjzFmSJSCewND
1004    spotify:track:0djZ2ndRfAL69WYNra5jRC
1005                                   error
                        ...                 
1996    spotify:track:2LjkT4gu5wO4JdeEYl0fMY
1997    spotify:track:32GWSQbpg1Ea1MMvl8HN23
1998    spotify:track:2jgPmBI3gIZ5C8DBlmFFGT
1999    spotify:track:4PFhHd7DsNg0ke9Nmlxms5
2000    spotify:track:2bAD41U8RQh4Z7rJinqzNy
Name: spotifyId_man, Length: 1000, dtype: object

In [100]:
songs_1k_to_2k_ids.value_counts()

error                                   124
spotify:track:4XVfNwP88Hn9kSkXVnxj2S      3
spotify:track:2g6V8u3EoMA1TAPHtwrRHc      2
spotify:track:1MfuskV4ODUp14C2dnyPgW      2
spotify:track:7hf89cT5FEmLV5E9fjrjG7      2
                                       ... 
spotify:track:72HKgOTKNpp3glpdLZc1yo      1
spotify:track:5OGfBbWmRRkDiZiJbu5WIr      1
spotify:track:5Jz5POp67L1TSM9FmdZSdR      1
spotify:track:2lmMpvoIqZWWFBcKoQ8KSC      1
spotify:track:43vI4vyL0hRk4rAIeQlbax      1
Name: spotifyId_man, Length: 841, dtype: int64

In [106]:
# Reset track_df_2
# Create df to hold Spotify track features returned from function below
track_df_2 = pd.DataFrame(columns = ['id_searched', 'name', 'album', 'artist', 'release_date', 
                                   'duration_ms', 'explicit', 'popularity', 'acousticness', 
                                   'danceability', 'energy', 'instrumentalness', 'liveness', 
                                   'loudness', 'speechiness', 'tempo', 'time_signature', 'key',
                                   'mode', 'valence'])

In [109]:
# test success for first 1k before go through 19k again...
for song_id in songs_1k_to_2k_ids:
    if (song_id == 'error'):
        continue
    else:
        getTrackFeatures2(song_id)

In [110]:
track_df_2

,id_searched,name,album,artist,release_date,duration_ms,explicit,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,key,mode,valence
1,spotify:track:5YjujDEmrypAj85VhKW8E9,Appreciate Me,Appreciate Me,Rack$tar Drvco,2020-12-28,180346,True,28,0.023700,0.694,0.421,0,0.1430,-12.885,0.3420,128.944,5,9,1,0.0905
2,spotify:track:1HFguX7RTd1GYkZzmJMX39,Appreciating Nature,Spiritual Nature Rain,#Sleepy Rain,2021-05-21,150000,False,0,0.000802,0.259,0.274,0.967,0.4070,-28.765,0.0867,110.964,4,8,1,0.0387
3,spotify:track:0BHSEhacCgjzFmSJSCewND,Approaching Ends,Slow Waves,Submerse,2014-07-25,205736,False,24,0.219000,0.668,0.487,0.0458,0.1590,-8.409,0.0824,173.963,3,5,0,0.1380
4,spotify:track:0djZ2ndRfAL69WYNra5jRC,April Come She Will,Sounds Of Silence,Simon & Garfunkel,1966-01-17,109386,False,61,0.863000,0.489,0.186,0.000152,0.2000,-16.237,0.0290,102.245,4,9,1,0.3490
5,spotify:track:71wT7aMCFPYfzutF66OLac,Aquaman,TALKING IS HARD,WALK THE MOON,2014-12-02,240000,False,44,0.510000,0.630,0.772,2.26e-05,0.0881,-6.986,0.0297,99.964,4,1,1,0.7210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,spotify:track:2LjkT4gu5wO4JdeEYl0fMY,Blue (feat. Blue Ivy),BEYONCÉ [Platinum Edition],Beyoncé,2014-11-24,266920,False,52,0.541000,0.459,0.435,6.51e-05,0.1190,-10.139,0.0640,90.567,4,1,1,0.0631
873,spotify:track:32GWSQbpg1Ea1MMvl8HN23,Blue and Yellow,The Used,The Used,2002-06-25,201426,False,53,0.029700,0.565,0.867,0,0.1470,-5.497,0.0333,115.006,4,2,0,0.2590
874,spotify:track:2jgPmBI3gIZ5C8DBlmFFGT,Blue Eyed Soul - 2017 Remaster,A.M. (Deluxe Edition),Wilco,1995,244853,False,16,0.267000,0.594,0.582,0.00744,0.1760,-7.301,0.0309,135.726,4,2,1,0.3440
875,spotify:track:4PFhHd7DsNg0ke9Nmlxms5,Blue Eyes - Remastered 2003,Jump Up!,Elton John,1982-01-01,206266,False,55,0.575000,0.518,0.229,0.00363,0.1230,-17.180,0.0270,106.665,3,7,0,0.1590


In [111]:
track_df_1k_to_2k = track_df_2.copy()

In [112]:
len(track_df_1k_to_2k)

876

In [113]:
track_df_1k_to_2k[track_df_1k_to_2k['id_searched'].duplicated(keep=False)].sort_values(by='id_searched')
# Duplicated Ids...
# Don't Dedupe until all are done

,id_searched,name,album,artist,release_date,duration_ms,explicit,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,key,mode,valence
6,spotify:track:0HZlND4giwzgolBpaNIRGV,Aquarius/Let The Sunshine In (The Flesh Failur...,The Age Of Aquarius,The 5th Dimension,1969,289293,False,59,0.44600,0.293,0.740,0.00151,0.0998,-6.098,0.0399,118.230,4,7,1,0.516
7,spotify:track:0HZlND4giwzgolBpaNIRGV,Aquarius/Let The Sunshine In (The Flesh Failur...,The Age Of Aquarius,The 5th Dimension,1969,289293,False,59,0.44600,0.293,0.740,0.00151,0.0998,-6.098,0.0399,118.230,4,7,1,0.516
231,spotify:track:0SIhsJvOPXxEm6fr2wNzWN,Baby,Comenzando la semana,Various Artists,2021-05-28,240013,False,0,0.16300,0.712,0.682,0,0.1130,-5.852,0.0556,160.494,4,7,1,0.484
204,spotify:track:0SIhsJvOPXxEm6fr2wNzWN,Baby,Comenzando la semana,Various Artists,2021-05-28,240013,False,0,0.16300,0.712,0.682,0,0.1130,-5.852,0.0556,160.494,4,7,1,0.484
820,spotify:track:0UM8xa2o8jHCpvqQEq6Rfw,Blame (feat. John Newman) - Jacob Plant Remix,Blame (Remixes) (feat. John Newman),Calvin Harris,2014-11-10,296106,False,39,0.00269,0.653,0.893,0.000622,0.6190,-3.869,0.0347,127.974,4,0,0,0.566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,spotify:track:7aLemVaGw5bt4u7wPXm7Lm,As Your Friend,As Your Friend,Afrojack,2013-01-01,240004,True,46,0.04670,0.654,0.761,0,0.1040,-4.404,0.0678,128.033,4,6,1,0.805
572,spotify:track:7g4I83EkCeMo779gSo4zKW,"Beethoven: Symphony No. 5 in C Minor, Op. 67: ...",50 Best Classics,Various Artists,2010-02-26,395680,False,41,0.98300,0.328,0.232,0.843,0.2180,-15.058,0.0444,85.821,4,0,0,0.201
570,spotify:track:7g4I83EkCeMo779gSo4zKW,"Beethoven: Symphony No. 5 in C Minor, Op. 67: ...",50 Best Classics,Various Artists,2010-02-26,395680,False,41,0.98300,0.328,0.232,0.843,0.2180,-15.058,0.0444,85.821,4,0,0,0.201
467,spotify:track:7hf89cT5FEmLV5E9fjrjG7,beamer boy,beamer boy,Lil Peep,2017-06-08,203373,True,77,0.05090,0.743,0.732,1.77e-05,0.2270,-4.511,0.1040,123.994,4,8,1,0.339


#### Create function to do this on repeat

In [72]:
# def get_song_id_and_feats(song_series): # Song Series must be labelled 'song'
#     song_df = song_series.to_frame() # Take Series and format as DataFrame
#     song_df['spotifyId_man'] = song_df['song'].map(lambda a: getSongId2(a)) # Create new column to capture spoitfy song Id
#     song_ids = song_df['spotifyId_man']
#     for id in song_ids:
#         if (song_id == 'error'):
#             continue
#         else:
#             getTrackFeatures2(song_id)

In [73]:
# test_function_1k_to_2k = leftover_songs_df['song'][1001:2001]

In [74]:
# get_song_id_and_feats(test_function_1k_to_2k)
# did not work correctly

In [ ]:
leftover_songs_df['song'][1001:2001]

for num in range(2,11):
    search_start = ((num-1)*1000)+1
    search_end = (num*1000)+1
    print(f'{search_start}:{search_end})

In [178]:
# for num in range(2,11):
#     search_start = ((num-1)*1000)+1
#     search_end = (num*1000)+1
#     leftover_songs_df['spotifyId_man'][search_start:search_end] = leftover_songs_df['song'][search_start:search_end].map(lambda a: getSongId(a))
#     print(f'{num} [{search_start}:{search_end}]')

In [179]:
# leftover_songs_df['spotifyId_man'][1001:2001] = leftover_songs_df['song'][1001:2001].map(lambda a: getSongId(a))
# songs_1k_to_2k = leftover_songs_df[1001:2001]

In [180]:
# leftover_songs_df['spotifyId_man'][2001:3001] = leftover_songs_df['song'][2001:3001].map(lambda a: getSongId(a))
# songs_2k_to_3k = leftover_songs_df[2001:3001]

In [181]:
# leftover_songs_df['spotifyId_man'][3001:4001] = leftover_songs_df['song'][3001:4001].map(lambda a: getSongId(a))
# songs_3k_to_4k = leftover_songs_df[3001:4001]

In [182]:
# leftover_songs_df['spotifyId_man'][4001:5001] = leftover_songs_df['song'][4001:5001].map(lambda a: getSongId(a))
# songs_4k_to_5k = leftover_songs_df[4001:5001]

In [183]:
# frames = [first_1k, songs_1k_to_2k, songs_2k_to_3k, songs_3k_to_4k, songs_4k_to_5k]
# first_5k_songs = pd.concat(frames)

In [184]:
# first_5k_songs

In [185]:
# leftover_songs_df['spotifyId_man'][5001:6001] = leftover_songs_df['song'][5001:6001].map(lambda a: getSongId(a))
# songs_5k_to_6k = leftover_songs_df[5001:6001]

In [186]:
# leftover_songs_df['spotifyId_man'][6001:7001] = leftover_songs_df['song'][6001:7001].map(lambda a: getSongId(a))
# songs_6k_to_7k = leftover_songs_df[6001:7001]

In [187]:
# leftover_songs_df['spotifyId_man'][7001:8001] = leftover_songs_df['song'][7001:8001].map(lambda a: getSongId(a))
# songs_7k_to_8k = leftover_songs_df[7001:8001]

In [188]:
# leftover_songs_df['spotifyId_man'][8001:9001] = leftover_songs_df['song'][8001:9001].map(lambda a: getSongId(a))
# songs_8k_to_9k = leftover_songs_df[8001:9001]

In [189]:
# leftover_songs_df['spotifyId_man'][9001:10001] = leftover_songs_df['song'][9001:10001].map(lambda a: getSongId(a))
# songs_9k_to_10k = leftover_songs_df[9001:10001]

In [190]:
# frames = [songs_5k_to_6k, songs_6k_to_7k, songs_7k_to_8k, songs_8k_to_9k, songs_9k_to_10k]
# second_5k_songs = pd.concat(frames)

In [191]:
# second_5k_songs

In [192]:
# frames = [first_5k_songs, second_5k_songs]
# first_10k_songs = pd.concat(frames)

In [193]:
# # pickle df - done
# first_10k_songs.to_pickle("./first_10k_songs.pkl")

In [194]:
# leftover_songs_df['spotifyId_man'][10001:11001] = leftover_songs_df['song'][10001:11001].map(lambda a: getSongId(a))
# songs_10k_to_11k = leftover_songs_df[10001:11001]

In [195]:
# leftover_songs_df['spotifyId_man'][11001:12001] = leftover_songs_df['song'][11001:12001].map(lambda a: getSongId(a))
# songs_11k_to_12k = leftover_songs_df[11001:12001]

In [196]:
# leftover_songs_df['spotifyId_man'][12001:13001] = leftover_songs_df['song'][12001:13001].map(lambda a: getSongId(a))
# songs_12k_to_13k = leftover_songs_df[12001:13001]

In [197]:
# leftover_songs_df['spotifyId_man'][13001:14001] = leftover_songs_df['song'][13001:14001].map(lambda a: getSongId(a))
# songs_13k_to_14k = leftover_songs_df[13001:14001]

In [198]:
# leftover_songs_df['spotifyId_man'][14001:15001] = leftover_songs_df['song'][14001:15001].map(lambda a: getSongId(a))
# songs_14k_to_15k = leftover_songs_df[14001:15001]

In [199]:
# frames = [songs_10k_to_11k, songs_11k_to_12k, songs_12k_to_13k, songs_13k_to_14k, songs_14k_to_15k]
# third_5k_songs = pd.concat(frames)

In [200]:
# third_5k_songs

In [201]:
# len(leftover_songs_df['spotifyId_man'])

In [202]:
# leftover_songs_df['spotifyId_man'][15001:16001] = leftover_songs_df['song'][15001:16001].map(lambda a: getSongId(a))
# songs_15k_to_16k = leftover_songs_df[15001:16001]

In [203]:
# leftover_songs_df['spotifyId_man'][16001:17001] = leftover_songs_df['song'][16001:17001].map(lambda a: getSongId(a))
# songs_16k_to_17k = leftover_songs_df[16001:17001]

In [204]:
# leftover_songs_df['spotifyId_man'][17001:18001] = leftover_songs_df['song'][17001:18001].map(lambda a: getSongId(a))
# songs_17k_to_18k = leftover_songs_df[17001:18001]

In [205]:
# leftover_songs_df['spotifyId_man'][18001:19001] = leftover_songs_df['song'][18001:19001].map(lambda a: getSongId(a))
# songs_18k_to_19k = leftover_songs_df[18001:19001]

In [206]:
# leftover_songs_df['spotifyId_man'][19001:19109] = leftover_songs_df['song'][19001:19109].map(lambda a: getSongId(a))
# songs_19k_to_end = leftover_songs_df[19001:19109]

In [207]:
# frames = [songs_15k_to_16k, songs_16k_to_17k, songs_17k_to_18k, songs_18k_to_19k, songs_19k_to_end]
# last_5k_songs = pd.concat(frames)

In [208]:
# frames = [third_5k_songs, last_5k_songs]
# last_10k_songs = pd.concat(frames)

In [209]:
# # pickle df - done
# last_10k_songs.to_pickle("./last_10k_songs.pkl")

In [210]:
# frames = [first_10k_songs, last_10k_songs]
# all_20k_songs = pd.concat(frames)
# all_20k_songs
# # really only 1908 rows

In [211]:
# # pickle df - done
# all_20k_songs.to_pickle("./all_20k_songs.pkl")

In [212]:
# leftover_songs_df = pd.read_pickle("./all_20k_songs.pkl")

In [213]:
# leftover_songs_df

In [214]:
# leftover_songs_df[leftover_songs_df['spotifyId_man']=='error']
# # 88.3% songs matched in leftovers

In [215]:
# song_class_count_spotid

In [216]:
# # Prep from merge rename column "song" with song title values as name
# leftover_songs_df.rename(columns={'song':'name'}, inplace=True)

In [217]:
# song_class_count_spotid.drop(columns='_merge', inplace=True)

In [218]:
# # repeat left join between original song_class_count_spotid and new leftover_songs_df based on song name
# song_class_count_spotid_2 = song_class_count_spotid.merge(leftover_songs_df, on='name', how='left', indicator=True)

In [219]:
# # Duplicates added, 9 dupes
# song_class_count_spotid_2['_merge'].value_counts()

In [220]:
# # View duplicate artists
# song_class_count_spotid_2[song_class_count_spotid_2['name'].duplicated(keep='first')].sort_values(by='name')['name']

In [221]:
# # Drop duplicate rows, take first spotify id matched with for 7 records
# song_class_count_spotid_2.drop_duplicates(subset='name', keep='first', inplace=True)

In [222]:
# # Reset index post drops
# song_class_count_spotid_2 = song_class_count_spotid_2.reset_index(drop=True)

In [223]:
# song_class_count_spotid_2

In [224]:
# # Replace nans in ['id'] with values found in manual Id pull (['spotifyId_man'])
# song_class_count_spotid_2['id'] = np.where(~song_class_count_spotid_2['id'].isnull(),
#                                            song_class_count_spotid_2['id'],song_class_count_spotid_2['spotifyId_man'])

In [225]:
# # Clean df by dropping 'spotifyId_man', '_merge' helper columns
# song_class_count_spotid_2.drop(columns=['spotifyId_man', '_merge'], inplace=True)

In [226]:
# song_class_count_spotid_2['id'].isnull().sum()

In [227]:
# 1-((song_class_count_spotid_2['id'].isnull().sum())/len(song_class_count_spotid_2))
# # 97% of songs have spotify Id - THIS STILL HAS ERRORS INCLUDED
# # 9

In [228]:
# len(song_class_count_spotid_2)

In [229]:
# song_class_count_spotid_2.describe().apply(lambda s: s.apply('{0:.5f}'.format))

In [230]:
# # pickle df - done
# song_class_count_spotid_2.to_pickle("./song_id_class_count.pkl")

In [231]:
# Bring in songs with track IDs table
song_id_class_count = pd.read_pickle("./song_id_class_count.pkl")

In [232]:
song_id_class_count[(song_id_class_count['id'] == 'error') | (song_id_class_count['id'].isnull())]

,name,Class Count,id,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Let's Stay Together - 2015 Remastered Version,1,error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Ready For Your Love (CLOSE Ready For Your Dub)...,1,error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,Baby (Single Version),1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,I Will Always Love You (Hex Hector Radio Mix: ...,11,error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29857,Now You See It (Benny Benassi Remix Radio Edit...,2,error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29868,L. 75: III. Clair de lune,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29896,Heard 'Em Say (feat. Adam Levine),28,error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29899,Wish You Were Here (feat. Brandyn Burnette) [E...,1,error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
